imports
-

In [1]:
import sys, os
sys.version

'3.9.13 (main, Aug 25 2022, 23:26:10) \n[GCC 11.2.0]'

In [2]:
#math
import numpy as np
#capture
import cv2
import wave
#utils
from tqdm import tqdm
from IPython.display import Image
import datetime

In [3]:
#disable CUDA devices
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [4]:
%load_ext tensorboard

In [5]:
#neural
import tensorflow as tf
import keras
from tensorflow.python.client import device_lib
from tensorflow.keras import layers
print(tf.__version__)

2.9.2


system info
-

In [6]:
for i in device_lib.list_local_devices():
    print(f'{i.device_type}|{i.name} memory limit: {i.memory_limit}')
    print(i.physical_device_desc)

CPU|/device:CPU:0 memory limit: 268435456



2022-10-15 03:17:02.204353: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-15 03:17:02.211222: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-10-15 03:17:02.211261: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: A-WS-M21
2022-10-15 03:17:02.211270: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: A-WS-M21
2022-10-15 03:17:02.211345: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.76.0
2022-10-15 03:17:02.211371: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 515.7

import data
-

In [7]:
def reshape(frame):
    global WIDTH, HEIGHT
    newframe = np.zeros(WIDTH*HEIGHT).reshape(HEIGHT, WIDTH)
    if frame.shape[0] == HEIGHT and frame.shape[1] == WIDTH:
        return frame
    elif frame.shape[0] <= HEIGHT and frame.shape[1] <= WIDTH:
        newframe[:frame.shape[0],:frame.shape[1]] = frame
    elif frame.shape[0] <= HEIGHT:
        newframe[:frame.shape[0],:] = frame[:, :WIDTH]
    elif frame.shape[1] <= WIDTH:
        newframe[:,:frame.shape[1]] = frame[:HEIGHT, :]
    else:
        newframe = frame[:HEIGHT, :WIDTH]
    return newframe

def deinterlase(frame):
    even_fields=frame[::2,:];
    odd_fields=frame[1::2,:];
    return even_fields, odd_fields

In [8]:
WIDTH = int(720)
HEIGHT = int(625-1)

def normilizeFrame(frame):
    global WIDTH, HEIGHT
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #change dimentions(choose 1 of 2 options)
    #frame = cv2.resize(frame, (WIDTH, HEIGHT), interpolation=cv2.INTER_NEAREST)
    frame = reshape(frame)
    
    return deinterlase(frame)
    #return frame / 255.0

def retrieve():
    global WIDTH, HEIGHT
    if cap.grab():
        ret, frame = cap.retrieve()
        if not ret:
            print('error reading frame')
            return False, normilizeFrame(np.zeros(WIDTH*HEIGHT).reshape(WIDTH, HEIGHT))
        return True, normilizeFrame(frame)
    else:
        return False, normilizeFrame(np.zeros(WIDTH*HEIGHT).reshape(WIDTH, HEIGHT))

def getNext():
    global first, topFrame, bottomFrame, half
    if first == True:
        first = False
        ret, (topFrame, bottomFrame) = retrieve()
        return ret, cv2.vconcat([topFrame, bottomFrame])[np.newaxis is None,:,:]
    
    if half:
        currTop = bottomFrame
        ret, (topFrame, bottomFrame) = retrieve()
        half = False
        return ret, cv2.vconcat([currTop, topFrame])[np.newaxis is None,:,:]
    else:
        half = True
        return True, cv2.vconcat([topFrame, bottomFrame])[np.newaxis is None,:,:]
    
fileName = 'testfiles/pal/whitenosie'

def openCap(file):
    global cap, gray, half, first
    half = True
    first = True
    cap = cv2.VideoCapture(file + '.avi')
    if (cap.isOpened()== False): 
      print("Error opening video stream or file")

openCap(fileName)

In [9]:
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(frameCount)

#tech calc

#audio in pcm have 44100 samples in second per channel, respectively we have 88200 samples
#per frame we have 44100/25(pal) = 1764 samples(for single channel)(and we take half of frame for learning /2)

samplesPerFrame = int(44100/cap.get(cv2.CAP_PROP_FPS)/2)
print(samplesPerFrame)

7501
882


create dataset

In [10]:
dir = "./dataset/"

In [11]:
mainAudioFile = wave.open(f'{dir}audio.wav', mode='r')

def gen():
    global mainAudioFile, samplesPerFrame
    currentFrame = 1
    
    #audio
    mainAudioFile.rewind()
    
    if not (mainAudioFile.getparams()[0] == 2 and mainAudioFile.getparams()[2] == 44100):
        print('wrong audiofile params')
        sys.exit(1)
    
    #video
    
    openCap(fileName)
    frames = cap.get(cv2.CAP_PROP_FRAME_COUNT) -1
    
    while currentFrame < frames:
        #audio
        frame = mainAudioFile.readframes(samplesPerFrame)
        
        if len(frame) < samplesPerFrame*2:
            frame = frame + b'\x00' * ((samplesPerFrame*4)-len(frame))
        
        #video
        vframe = getNext()[1]
        
        yield vframe, tf.io.decode_raw(frame, tf.int16, little_endian=True).numpy()/16383.0
        currentFrame = currentFrame+1
    

In [12]:
dataset=tf.data.Dataset.from_generator(
    gen, 
    output_signature=(tf.TensorSpec(shape=(1, HEIGHT, WIDTH), dtype=tf.uint8, name='image'),
                      tf.TensorSpec(shape=(samplesPerFrame*2,), dtype=tf.int16, name='audio'),
                      #tf.TensorSpec(shape=(1), dtype=tf.int32, name='frameNum')
                     )
)#.cache('./dataset/cache/')#.repeat()

#create model
-

In [13]:
input = keras.Input(shape=((HEIGHT), WIDTH), name = 'image')

In [14]:
input.shape

TensorShape([None, 624, 720])

In [15]:
x = tf.keras.layers.Rescaling(1./127.5, name='normalize', offset=-1)(input)
x = tf.keras.layers.Reshape((x.shape[1], x.shape[2], 1))(x)
filter = 8
sizekern = 64

i=1
x = tf.keras.layers.Conv2D(filter, (4, sizekern), activation='relu', name=f'conv2d_{i}')(x)
x = tf.keras.layers.MaxPooling2D(name=f'MaxPooling2D_{i}')(x)

filter = 1
sizekern = 256

i=2
x = tf.keras.layers.Conv2D(filter, (2, sizekern), activation='relu', name=f'conv2d_{i}')(x)
x = tf.keras.layers.MaxPooling2D(name=f'MaxPooling2D_{i}')(x)

In [16]:
print(x.shape)
print(x.shape[1]*x.shape[2]*x.shape[3])
print(f'diff {x.shape[1]*x.shape[2]*x.shape[3]-(samplesPerFrame*2)}')

(None, 154, 36, 1)
5544
diff 3780


In [17]:
x = tf.keras.layers.Flatten(name='Flatten')(x)

In [18]:
#x = tf.keras.layers.Dense((samplesPerFrame*2), activation='linear', bias_initializer='ones', use_bias=True, name='layer_1')(x)

In [19]:
output = tf.keras.layers.Dense((samplesPerFrame*2), activation='sigmoid', bias_initializer='ones', use_bias=True, name='audio')(x)

In [20]:
model = keras.Model(input, output)
model.summary()

#tf.keras.utils.plot_model(
#    model,
#    to_file="model.png",
#    show_shapes=True,
#    show_dtype=False,
#    show_layer_names=True,
#    rankdir="TB",
#    expand_nested=True,
#    dpi=96,
#    layer_range=None,
#    show_layer_activations=True,
#)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 624, 720)]        0         
                                                                 
 normalize (Rescaling)       (None, 624, 720)          0         
                                                                 
 reshape (Reshape)           (None, 624, 720, 1)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 621, 657, 8)       2056      
                                                                 
 MaxPooling2D_1 (MaxPooling2  (None, 310, 328, 8)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 309, 73, 1)        4097      
                                                             

compile and learn
-

In [21]:
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['binary_accuracy'])

In [22]:
testfile = 'testfiles/pal/streetSrakerffV1'

In [23]:
!rm -rf ./logs/

In [24]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [25]:
loss = 1
step = 1
while loss >= 0.1:
    fitHistory = model.fit(dataset, epochs=2, callbacks=[tensorboard_callback])
    model.save("model")
    if step%1 == 0:
        openCap(testfile + '.avi')
        
        bar = tqdm(total=cap.get(cv2.CAP_PROP_FRAME_COUNT), ncols=100)
        file = wave.open(f'output/{step}.wav', mode='w')
        file.setnchannels(2) 
        file.setsampwidth(2)
        file.setframerate(44100)
        ret, vframe = getNext()
        while ret:
            file.writeframes((model.predict(vframe, verbose=0)[0]*16383).astype(np.int16).tobytes())
            bar.update(1)
            ret, vframe = getNext()
            if int(cap.get(cv2.CAP_PROP_POS_FRAMES)) > 25*2*15:    #limiting 30 on seconds
                ret = False
        bar.close()
        file.close()
        step = step +1
    loss = fitHistory.history['loss'][0]

Epoch 1/5
   7499/Unknown - 7943s 1s/step - loss: 0.5003 - binary_accuracy: 0.5000

2022-10-15 05:29:26.091845: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2347107840 exceeds 10% of free system memory.


7499/7499 [==============================] - 7944s 1s/step - loss: 0.5003 - binary_accuracy: 0.5000
Epoch 2/5
7499/7499 [==============================] - ETA: 0s - loss: 0.5002 - binary_accuracy: 0.5001

2022-10-15 07:40:34.860680: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2347107840 exceeds 10% of free system memory.


7499/7499 [==============================] - 7869s 1s/step - loss: 0.5002 - binary_accuracy: 0.5001
Epoch 3/5
7499/7499 [==============================] - ETA: 0s - loss: 0.5002 - binary_accuracy: 0.5001

2022-10-15 09:52:47.388609: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2347107840 exceeds 10% of free system memory.


7499/7499 [==============================] - 7933s 1s/step - loss: 0.5002 - binary_accuracy: 0.5001
Epoch 4/5
7499/7499 [==============================] - ETA: 0s - loss: 0.5002 - binary_accuracy: 0.5001

2022-10-15 12:08:49.413714: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2347107840 exceeds 10% of free system memory.


7499/7499 [==============================] - 8162s 1s/step - loss: 0.5002 - binary_accuracy: 0.5001
Epoch 5/5
1005/7499 [===>..........................] - ETA: 1:58:10 - loss: 0.4997 - binary_accuracy: 0.5006


KeyboardInterrupt



In [37]:
cv2.imwrite('1.png', vframe.reshape(624, 720))

True

In [36]:
vframe.reshape(624, 720)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [30]:
%tensorboard --logdir logs/fit

save
-